In [1]:
from Bio.Blast import NCBIWWW

sequence_data = open("1seq.fasta").read()
sequence_data
result_handle = NCBIWWW.qblast("blastp", "nr",sequence_data, hitlist_size=500) 

with open('results.xml', 'w') as save_file: 
    blast_results = result_handle.read() 
    save_file.write(blast_results)


In [2]:
from Bio.Blast import NCBIXML
import pandas as pd

    
result=open("cres.xml","r")
records= NCBIXML.parse(result)
item=next(records)
result.close()
E_VALUE_THRESH = 0.04

for alignment in item.alignments:
    f= open('humanparse.xml', 'a')
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            
            f.write("\n"+" ****Alignment****"+"\nsequence: "+alignment.title+"\nlength: "+str(alignment.length)+"\ne value:"+ str(hsp.expect)+"\naligned query: \n"+hsp.query[0:alignment.length]+"\n")
f.close()
            


In [6]:
#gcf is the number of alignments so in this case we took 100 alignments for all results
gcf=100
indcnt=0
x=[]
cnt=0
import pandas as pd
df_all=pd.DataFrame()
newdf=pd.DataFrame()
secdf=pd.DataFrame()
dtf=49500
def getfinallist():
    import xlrd
    import xlsxwriter
    import re
    from itertools import takewhile
    
    ntermprtn=[]  
    indexPosList = []
    
    def column_len(sheet, index):
        col_values = sheet.col_values(index)
        col_len = len(col_values)
        for _ in takewhile(lambda x: not x, reversed(col_values)):
            col_len -= 1
        return col_len

    name1=str(dtf)+"_df"+".xlsx"                                    
    wb2 = xlrd.open_workbook(name1) 
    sheet = wb2.sheet_by_index(0)
    
    
    for i in range(0, sheet.nrows):
        row = sheet.row_slice(i)
        ntermprtn.append(row[0].value)
    
    k=0
    
    for i in ntermprtn:
        k=k+1
        if i:
            n=k
            indexPosList.append(n)
    
    return indexPosList

list1=getfinallist()
#print(list1)


def ncterminal(index1,index_sec,cnt,dtf):

    import xlrd
    import xlsxwriter
    import re
    from Bio.Blast import NCBIXML
    #To store the c and n terminal of all the fusion proteins in 2 lists
    ntermprtn=[]  
    ctermprtn=[]
    start=[]
    end=[]
    defn=[]
    indexPosList = []
    seqence=[]
    
    name1=str(dtf)+"_df"+".xlsx" 

    #get the parsed data                                         
    wb2 = xlrd.open_workbook(name1) 
    sheet = wb2.sheet_by_index(0) 
       
    for i in range(index1-1,index_sec-1):
        ntermprtn.append(sheet.cell_value(i,0))
    for i in range(index1-1,index_sec-1): 
        ctermprtn.append(sheet.cell_value(i,1))
    for i in range(index1-1,index_sec-1): 
        seqence.append(sheet.cell_value(i,2))
    for i in range(index1-1,index_sec-1): 
        start.append(sheet.cell_value(i,3))
    for i in range(index1-1,index_sec-1): 
        end.append(sheet.cell_value(i,4))
    for i in range(index1-1,index_sec-1): 
        defn.append(sheet.cell_value(i,5))
   
    
    print(ntermprtn)
    #will have to loop for multiple proteins
    patc=ctermprtn[0]
    
    #creating dictionary to understand the alignment definition line
    excel_file = 'gene_dictionary.xlsx'
    book = xlrd.open_workbook(excel_file)
    sheet = book.sheet_by_index(0)
    ensembl1=0
    gene1=1
    gene_names1=2 
    gene_desc1 = 3
    uniprot1 = 4
    gene_syn1=5

    gene_syn=[]
    gene_names=[]
    gene_desc=[]
    uniprot=[]
    ensembl=[]
    gene=[]

    for row in range(sheet.nrows):
        gene_syn.append(sheet.cell(row,gene_syn1).value) 
        gene_desc.append(sheet.cell(row,gene_desc1).value)
        gene_names.append(sheet.cell(row,gene_names1).value)
        uniprot.append(sheet.cell(row,uniprot1).value)
        ensembl.append(sheet.cell(row,ensembl1).value)
        gene.append(sheet.cell(row,gene1).value)

    gsc=[] 
    gdc=[]
    upc=[]
    enc=[]
    gname=[]
    gec=[s for s in gene if patc in s]
    


    if gec != []:
        element=gec[0]
        indexdict=gene.index(element)
    else:
        print('Error: Dictionary does not have C terminal protein: '+ patc)

    
    gdc=gene_desc[indexdict]
    gdc=gdc.split(",")
    gsc=gene_syn[indexdict]
    gsc=gsc.split(",")
    upc.append(uniprot[indexdict])
    enc=ensembl[indexdict]
    enc=enc.split(";")
    enc=list(filter(None, enc))
    gname=gene_names[indexdict]
    #print(gname)
    gname=gname.split(",")
    

    i=0


    matchc=[]
    matchc_al=[]

    h=0

    for i in defn:
        titlestr=defn[h]
        if gsc != []:
            if titlestr.find(gsc[0]) != -1:
                matchc.append(defn.index(titlestr))
                
        if gdc != []:
            if titlestr.find(gdc[0]) != -1:
                matchc.append(defn.index(titlestr))
                
        if upc != []:
            if titlestr.find(upc[0]) != -1:
                matchc.append(defn.index(titlestr))
                
        if enc != []:
            if titlestr.find(enc[0]) != -1:
                matchc.append(defn.index(titlestr))
                
        if gec != []:
            if titlestr.find(gec[0]) != -1:
                matchc.append(defn.index(titlestr))
                
        if gname != []:
            if titlestr.find(gname[0]) != -1:
                matchc.append(defn.index(titlestr))
                
                
        h=h+1
    
    #finalc = [s for s in defn if pat in s]
    #To get the hit definition and match it for homo sapiens and the n terminal protein
    #patn="none"
    
    patn=ntermprtn[0] #will have to loop for multiple proteins
    #matchhsn = [s for s in defnn if "Homo sapiens" in s]
    #finaln = [s for s in defn if pat in s]
    
 

    gsn=[] 
    gdn=[]
    upn=[]
    enn=[]
    gnamen=[]
    gen=[s for s in gene if patn in s]
    
    
    
    if gen != []:
        element=gen[0]
        
        indexdictn=gene.index(element)
        
        
    else:
        print('Error: The dictionary does not contain N terminal protein:'+ patn)
    
    

   
    
    gdn.append(gene_desc[indexdictn])
    gsn=gene_syn[indexdictn]
    gsn=gsn.split(",")
    upn.append(uniprot[indexdictn])
    enn=ensembl[indexdictn]
    enn=enn.split(";")
    enn=list(filter(None, enn))
    gnamen=gene_names[indexdictn]
    gnamen=gnamen.split(",")


    matchn=[]
    matchn_al=[]

    h=0

    for i in defn:
        titlestr=defn[h]
        if gsn != []:
            if titlestr.find(gsn[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
        if gdn != []:
            if titlestr.find(gdn[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
        if upn != []:
            if titlestr.find(upn[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
        if enn != []:
            if titlestr.find(enn[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
        if gen != []:
            if titlestr.find(gen[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
        if gnamen != []:
            if titlestr.find(gnamen[0]) != -1:
                indn=defn.index(titlestr)
                matchn.append(indn)
                
        h=h+1

    #print('This is matchn:')
    #print(matchn)

    #store the indices of the matches in a list


    end_match_c=[]
    end_match_n=[]
    start_match_n=[]
    start_match_c=[]
    im=0
    ik=0

    while ik < len(matchc):   
        end_match_c.append(end[matchc[ik]])
        ik=ik+1
    
    while im < len(matchn):   
        end_match_n.append(end[matchn[im]])
        im=im+1
    ik=0
    im=0
    while ik < len(matchc):   
        start_match_c.append(start[matchc[ik]])
        ik=ik+1
    while im < len(matchn):   
        start_match_n.append(start[matchn[im]])
        im=im+1

    import numpy as np
    import pandas as pd

    fusprtn=[]
    fusprtn.append(patn+"/"+patc)
    sq=seqence[0]
    
    df = pd.DataFrame({'Protein': pd.Series(fusprtn),'Sequence': pd.Series(sq), 'Start N': pd.Series(start_match_n), 'End N': pd.Series(end_match_n), 'Start C': pd.Series(start_match_c), 'End C': pd.Series(end_match_c)})
    
    return indexPosList, df, end_match_n, start_match_c,fusprtn,sq



def get_linker(fp,sq,mn,en,st):
    enn=en
    #print(en)
    stc=st
    seq = sq
    prt = fp
    
    
    mx1=len(enn)
    mx2=len(stc)
    linklistlen=[]
    linklist=[]
    nloc=[]
    cloc=[]
    if mx1==0 or mx2==0:
        nloc.append('none')
        cloc.append('none')
        linklistlen.append('no linker region found')
    if mx1<mx2:
        sm=enn
        bh=stc
    else:
        sm=stc
        bh=enn
    jm=0
    jk=0
    #not sure if third condition is okay check with maam
    if sm == enn:
        for jm in sm:
            for jk in bh:
                if jm>jk or jm==jk:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                elif (jk-jm) == 1:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                elif (jk-jm) >= 90:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                else:
                    nloc.append(jm)
                    cloc.append(jk)
                    linklistlen.append(jk-jm)
                    
    else:
        for jm in sm:
            for jk in bh:
                if jm<jk or jm==jk:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                elif (jm-jk) == 1:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                elif (jm-jk) >= 90:
                    nloc.append('none')
                    cloc.append('none')
                    linklistlen.append('no linker region found')
                else:
                    nloc.append(jk)
                    cloc.append(jm)
                    linklistlen.append(jm-jk)
    
    a=0
    b=0
    lin=[]
    nind=0
    
    for a in nloc:
        if a=='none' or cloc[nind]=='none':
            lin.append('no linker')
            
        else:
            lin.append(sq[int(a):int(cloc[nind])])
           
        nind+=1
    
    
    df2=pd.DataFrame({'Protein': pd.Series(fp),'Sequence': pd.Series(seq),'Linker Length': pd.Series(linklistlen), 'N term': pd.Series(nloc), 'C term': pd.Series(cloc), 'Linker': pd.Series(lin)})                  
    
    print(df2)
    return df2


                                        

#print(seqlist)
#list1.pop(0)

mn=1
s=[]
inde1=0
index1=0
inde2=0
index2=0
for i in list1:
    if i == list1[len(list1)-1]:
        break
    #print(i)
    inde1=list1.index(i)
    index1=list1[inde1]
    
    #print(index1)
    inde2=inde1+1
    
    index_sec=list1[inde2]
    
    
    (pos,l,strpt,endpt,fp,sq) = ncterminal(index1,index_sec,cnt,dtf)
    
    df_all= pd.concat([df_all], axis=0)
    newdf=pd.concat([newdf,l], axis=0)
    cnt=cnt+1
    (df_2)=get_linker(fp,sq,mn,strpt,endpt)
    
    secdf=pd.concat([secdf,df_2], axis=0)
    mn=mn+1
    
    
    
print(list1)

name2="res_"+str(dtf)+".xlsx" 
name3="linker_"+str(dtf)+".xlsx" 
newdf.to_excel(name2, index=False)
secdf.to_excel(name3, index=False)


    



['TBC1D22A', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
             Protein                                           Sequence  \
0   TBC1D22A/ARFGAP3  MASDGARKQFWKRSNSKLPGSLLRSTAKMPTTPVKAKRVSTFQEFE...   
1                NaN                                                NaN   
2                NaN                                                NaN   
3                NaN                                                NaN   
4                NaN                                                NaN   
..               ...                                                ...   
79               NaN                                                NaN   
80               NaN                                                NaN   
81               NaN                                                NaN   
82               

['DROSHA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0  DROSHA/TERT  MMQGNTCHRMSFHPGRGCPRGRGGHGARPSAPSFRPQNLRLLHPQQ...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['DROSHA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0  DROSHA/TERT  MMQGNTCHRMSFHPGRGCPRGRGGHGARPSAPSFRPQNLRLLHPQQ...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['DROSHA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0  DROSHA/TERT  MMQGNTCHRMSFHPGRGCPRGRGGHGARPSAPSFRPQNLRLLHPQQ...   

            Linker Length N term C term     Linker  
0  no lin

['NOTCH2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
        Protein                                           Sequence  \
0   NOTCH2/REG4  MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...   
1           NaN                                                NaN   
2           NaN                                                NaN   
3           NaN                                                NaN   
4           NaN                                                NaN   
5           NaN                                                NaN   
6           NaN                                                NaN   
7           NaN                  

      Protein                                           Sequence  \
0  HEXA/MYO9A  MTSSRLWFSLLLAAAFAGRATALWPWPQNFQTSDQRYVLYPNNFQF...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['HEXA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
      Protein                                           Sequence  \
0  HEXA/MYO9A  MTSSRLWFSLLLAAAFAGRATALWPWPQNFQTSDQRYVLYPNNFQF...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['HEXA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
      Protein                                           Sequence  \
0  HEXA/MYO9A  MTSSRLWFSLLLAAAFAGRATALWPWPQNFQTSDQRYVLYPNNFQF...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['HEXA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    

['GNS', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0   GNS/RAB3IP  MDNYLLLQTPLKKTKALIGEMGMTFSSAYVPSALCCPSRASILTGK...   
1          NaN                                                NaN   
2          NaN                                                NaN   
3          NaN                                                NaN   
4          NaN                                                NaN   
..         ...                                                ...   
58         NaN                                                NaN   
59         NaN                              

...

['ZNF695', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
        Protein                                           Sequence  \
0  ZNF695/SMYD3  MVICNSFTICNAEMQEVGVGLYPSISLLNHSCDPNCSIVFNGPHLL...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['ZNF695', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

['JDP2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
      Protein                                           Sequence  \
0  JDP2/YLPM1  MMPGQIPDPSVTTGSLPGLGPLTGLPSSALTVEELKYADIRNLGAM...   
1         NaN                                                NaN   
2         NaN                                                NaN   
3         NaN                                                NaN   
4         NaN                                                NaN   
5         NaN                                                NaN   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
1  no linker region found   none   none  no linker  
2  no linker region found   none   none  no linker  
3  no linker region found   none   none  no linker  
4  no linker region found   none   none  no linker  
5  no linker region found   none   none  no linker  
['JDP2', '', '', '', '', '', '', '', '', '', '', '', ''

        Protein                                           Sequence  \
0  SLC5A1/YWHAH  MPEYLRKRFGGQRIQVYLSLLSLLLYIFTKISADIFSGAIFINLAL...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['THSD4', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
        Protein                                           Sequence  \
0  THSD4/ADAM10  MVSHFMGSLSVLCFLLLLGFQFVCPQPSTQHRKVPQRMAAEGAPED...   
1           NaN                                                NaN   
2           NaN                                                NaN   
3           NaN                                                NaN 

['TYW1', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0   TYW1/TRA2A  MDPSADTWDLFSPLISLWINRFYIYLGFAVSISLWICVQIVIKTQG...   
1          NaN                                                NaN   
2          NaN                                                NaN   
3          NaN                                                NaN   
4          NaN                                                NaN   
5          NaN                                                NaN   
6          NaN                                                NaN   
7          NaN                             

       Protein                                           Sequence  \
0  ANKS3/GLYR1  MYASYIGHDTIVHLLLEAGVSVNVPTPEGQTPLMLASSCGNESIAY...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['ANKS3', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
       Protein                                           Sequence  \
0  ANKS3/GLYR1  MYASYIGHDTIVHLLLEAGVSVNVPTPEGQTPLMLASSCGNESIAY...   

            Linker Length N term C term     Linker  
0  no linker region found   none   none  no linker  
['ANKS3', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''